In [30]:
from transformers import BertForSequenceClassification

sothebysBERT = BertForSequenceClassification.from_pretrained('../Homework-Notebooks/week-8/SothebysBERT/')

In [31]:
import pandas as pd
import numpy as np
import torch

In [32]:
sothebysUS = pd.read_pickle("./sothebysUS_noRawImg.pkl")

In [33]:
sothebysUS.head(2)

,Unnamed: 0,Origin,CurrencyType,AuctionDate,LotNumber,SoldPrice,Collection,ItemTitle,EstPriceInUSD,AllText,LogSoldPrice,PriceBin,input_ids,attention_mask,PriceBin_OneHot,Image,SiamFeatures,cnnFeatures
0,1,Louis Hayet,USD,2015-01-30T10:00:00.935-0500,582,28125,N09303,"LE CONCERT, TOGETHER WITH THREE OTHER WORKS",28125,gouache on paper moun...,10.244414,1,"[101, 2175, 6692, 5403, 2006, 3259, 5614, 2006...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 1.0, 0.0]",N09303_582,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.16779256, 0.0, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,2,A light chestnut velvet-upholstered Knole sofa...,USD,2015-10-17T10:00:00.852-0400,807,5000,N09408,NaN,5000,height 45 1/2 in.; l...,8.517193,0,"[101, 4578, 3429, 1015, 1013, 1016, 1999, 1012...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 0.0, 0.0]",N09408_807,"[[0.0, 0.0, 0.21109672, 0.0, 0.0, 0.0, 0.0, 1....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [5]:
sothebysBERT.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [42]:
def getSenEmbed(text, mask, model, device):
    textEmb = torch.tensor([text]).to(device)
    attMask = torch.tensor([mask]).to(device)
    hidden_states = model(textEmb, attMask).hidden_states
    result = hidden_states[-2][0][:100]
    
    return result.to("cpu")

In [43]:
sothebysBERT.cuda()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [44]:
getSenEmbed(sothebysUS["input_ids"][0], sothebysUS["attention_mask"][0], sothebysBERT,device).detach().numpy().reshape(-1).shape

(76800,)

In [45]:
sothebysUS["BERTSenEmbed"] = sothebysUS[["input_ids","attention_mask"]].apply(lambda x: getSenEmbed(x["input_ids"],x["attention_mask"],sothebysBERT,device).detach().numpy().reshape(-1), axis=1)


In [1]:
import pandas as pd
sothebysUS = pd.read_pickle("./sothebysUS_fullFeatures.pkl")

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfVectorizer = TfidfVectorizer(max_df=0.5, max_features=2048, min_df=3, stop_words='english', norm='l2')


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sothebysUS[[
                                                               "BERTSenEmbed",
                                                               "SiamFeatures"]], 
                                                    sothebysUS["LogSoldPrice"], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [3]:
# preparing bert feature
bertMatrix = np.array(X_train["BERTSenEmbed"].tolist())


In [6]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2048)

In [4]:
bertMatrix.shape

(41319, 76800)

In [ ]:
# preparing Bert's feature
reduceBert = pca.fit_transform(bertMatrix)


In [ ]:
# preparing siamese + cnn feature


In [ ]:
# preparing BERT + Siamese feature


In [87]:
from sklearn.linear_model import RidgeCV

cnnRidge = RidgeCV(alphas=[1e-2, 1e-1, 1, 5, 10], cv=10).fit(X_train["cnnFeatures"], y_train)
cnnRidge = RidgeCV(alphas=[1e-2, 1e-1, 1, 5, 10], cv=10).fit(X_train["cnnFeatures"], y_train)

,AllText,BERTSenEmbed,SiamFeatures,cnnFeatures,PriceBin_OneHot,PriceBin
30735,comprising a flemish ...,"[[-3.460422, -5.3442187, -0.42756945, 0.009015...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 0.0, 0.0]",0
7374,signed w.a.margetson...,"[[-4.371092, -4.8330235, -1.4295642, -0.909767...","[[0.35328758, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 0.0, 0.0]",0
13062,oil on canvas 44 1/2 ...,"[[-2.872075, -1.2208447, -0.19730213, 0.189951...","[[0.0, 0.0, 0.0, 0.0, 0.3704357, 0.0, 0.0, 0.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 1.0, 0.0]",1
35025,height 20 in.; length...,"[[-5.6439047, -3.3894978, -0.18164915, 1.39331...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 0.0, 0.0]",0
30765,comprising: one rect...,"[[-6.0844073, -4.0902367, -0.6526308, 0.281285...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22251916, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 0.0, 0.0]",0
...,...,...,...,...,...,...
11284,"signed, with three se...","[[-4.2899632, -1.2532886, 1.963121, 0.26511422...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 1.0, 0.0]",1
44732,with a cartouche-sha...,"[[-3.1552248, -4.718044, -1.2360809, -0.473314...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 1.0, 0.0]",1
38158,autograph letter sign...,"[[-3.5943651, -2.101049, 1.686606, 0.29676402,...","[[0.0027186507, 0.0, 0.07416968, 0.0, 0.0, 1.9...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 0.0, 0.0]",0
860,"each, oil on paperboa...","[[-4.4474196, -2.3365483, 0.3526125, 0.3142795...","[[0.0, 0.5087738, 0.0, 0.0, 0.0, 0.0, 2.230557...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 1.0, 0.0]",1
